# **Homework 2-1 Phoneme Classification**

Author: Chih-Yuan Chuang (r09921006)

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/windows備份/C_Users_Alvin_Documents/~碩班/Course/109-2/機器學習_李宏毅/Week 3/HW2

Mounted at /content/drive


In [ ]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!cp '/content/drive/MyDrive/windows備份/C_Users_Alvin_Documents/~碩班/Course/109-2/機器學習_李宏毅/Week 3/HW2/timit_11_v2.zip' './data.zip'
!unzip data.zip
!ls 

Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  drive  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [ ]:
train_mean = np.mean(train, axis=0, keepdims=True)
train_std = np.std(train, axis=0, keepdims=True)
print('Size of training data mean: {}'.format(train_mean.shape))
print('Size of training data std: {}'.format(train_std.shape))

Size of training data mean: (1, 429)
Size of training data std: (1, 429)


In [ ]:
# Normalization
# If "train = (train - train_mean) / train_std", memory usage is too large somehow???
train = (train - train_mean)
train = train / train_std
test = (test - train_mean)
test = test / train_std
print('Size of training data after normalization: {}'.format(train.shape))
print('Size of testining data after normalization: {}'.format(test.shape))
import gc
del train_mean, train_std
gc.collect()

Size of training data after normalization: (1229932, 429)
Size of testining data after normalization: (451552, 429)


61

## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int) # 數據轉換成int
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.1

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))


Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 128

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
#import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

152

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.hl = 512
        self.gru = nn.GRU(39, self.hl, 2, bidirectional=True)
        self.fc = nn.Linear(self.hl*11*2, 39)
    
    def forward(self, x):
        out = x.view(x.size(0), 11, 39) #128*11*39
        out = out.permute(1, 0, 2) #11*128*39
        h0 = Variable(torch.zeros(2*2, x.size(0), self.hl).cuda())
        out, hn = self.gru(out, h0) # 11*128*512
        out = out.permute(1, 0, 2) 
        out = out.reshape(x.size(0), self.hl*11*2) #128*(5632*2)
        out = self.fc(out) 
        return out

## Training

In [ ]:
#check device
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
!pip install torchensemble
from torchensemble import VotingClassifier
from torch.autograd import Variable
from torchensemble.utils.logging import set_logger

     |████████████████████████████████| 22.3MB 135kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
logger = set_logger("hw2_log")
model = VotingClassifier(estimator=Classifier ,n_estimators=12, cuda=True).to(device)
optimizer = model.set_optimizer("Adam", lr=0.001, weight_decay=5e-5)
model.set_scheduler("CosineAnnealingLR", T_max=4)
model.fit(train_loader, epochs=4, test_loader=val_loader)

DEVICE: cuda
Log will be saved in '/content/logs'.
Start logging into file /content/logs/hw2_log-2021_04_02_12_06.log...
Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 3.67261 | Correct: 2/128
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 1.91726 | Correct: 50/128
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 1.48722 | Correct: 72/128
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 1.55723 | Correct: 66/128
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 1.34021 | Correct: 77/128
Estimator: 000 | Epoch: 000 | Batch: 500 | Loss: 1.21041 | Correct: 76/128
Estimator: 000 | Epoch: 000 | Batch: 600 | Loss: 1.39146 | Correct: 74/128
Estimator: 000 | Epoch: 000 | Batch: 700 | Loss: 1.26030 | Correct: 71/128
Estimator: 000 | Epoch: 000 | Batch: 800 | Loss: 1.14532 | Correct: 86/128
Estimator: 000 | Epoch: 000 | Batch: 900 | Loss: 1.11317 | Correct: 82/128
Estimator: 000 | Epoch: 000 | Batch: 1000 | Loss: 1.30320 | Correct: 79/128
Estimator: 000 | Epoch: 000 | Batch: 1100 | Loss: 1.02

2021-04-02 12:28:32,222 - INFO: Saving the model to `./VotingClassifier_Classifier_12_ckpt.pth`
2021-04-02 12:28:32,985 - INFO: Epoch: 000 | Validation Acc: 76.445 % | Historical Best: 76.445 %


Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 0.78051 | Correct: 95/128
Estimator: 000 | Epoch: 001 | Batch: 100 | Loss: 0.92085 | Correct: 90/128
Estimator: 000 | Epoch: 001 | Batch: 200 | Loss: 0.55271 | Correct: 107/128
Estimator: 000 | Epoch: 001 | Batch: 300 | Loss: 0.75484 | Correct: 93/128
Estimator: 000 | Epoch: 001 | Batch: 400 | Loss: 0.67581 | Correct: 95/128
Estimator: 000 | Epoch: 001 | Batch: 500 | Loss: 0.84126 | Correct: 89/128
Estimator: 000 | Epoch: 001 | Batch: 600 | Loss: 0.69933 | Correct: 92/128
Estimator: 000 | Epoch: 001 | Batch: 700 | Loss: 0.87351 | Correct: 88/128
Estimator: 000 | Epoch: 001 | Batch: 800 | Loss: 0.64340 | Correct: 100/128
Estimator: 000 | Epoch: 001 | Batch: 900 | Loss: 0.59572 | Correct: 98/128
Estimator: 000 | Epoch: 001 | Batch: 1000 | Loss: 0.55409 | Correct: 104/128
Estimator: 000 | Epoch: 001 | Batch: 1100 | Loss: 0.76489 | Correct: 95/128
Estimator: 000 | Epoch: 001 | Batch: 1200 | Loss: 0.61940 | Correct: 100/128
Estimator: 000 | E

2021-04-02 12:49:51,153 - INFO: Saving the model to `./VotingClassifier_Classifier_12_ckpt.pth`
2021-04-02 12:49:52,048 - INFO: Epoch: 001 | Validation Acc: 77.403 % | Historical Best: 77.403 %


Estimator: 000 | Epoch: 002 | Batch: 000 | Loss: 0.54659 | Correct: 103/128
Estimator: 000 | Epoch: 002 | Batch: 100 | Loss: 0.61792 | Correct: 106/128
Estimator: 000 | Epoch: 002 | Batch: 200 | Loss: 0.39378 | Correct: 109/128
Estimator: 000 | Epoch: 002 | Batch: 300 | Loss: 0.57608 | Correct: 109/128
Estimator: 000 | Epoch: 002 | Batch: 400 | Loss: 0.39553 | Correct: 109/128
Estimator: 000 | Epoch: 002 | Batch: 500 | Loss: 0.40070 | Correct: 112/128
Estimator: 000 | Epoch: 002 | Batch: 600 | Loss: 0.66806 | Correct: 94/128
Estimator: 000 | Epoch: 002 | Batch: 700 | Loss: 0.45775 | Correct: 102/128
Estimator: 000 | Epoch: 002 | Batch: 800 | Loss: 0.52904 | Correct: 105/128
Estimator: 000 | Epoch: 002 | Batch: 900 | Loss: 0.53583 | Correct: 104/128
Estimator: 000 | Epoch: 002 | Batch: 1000 | Loss: 0.41427 | Correct: 109/128
Estimator: 000 | Epoch: 002 | Batch: 1100 | Loss: 0.43203 | Correct: 110/128
Estimator: 000 | Epoch: 002 | Batch: 1200 | Loss: 0.48475 | Correct: 109/128
Estimator:

2021-04-02 13:11:10,018 - INFO: Saving the model to `./VotingClassifier_Classifier_12_ckpt.pth`
2021-04-02 13:11:10,778 - INFO: Epoch: 002 | Validation Acc: 77.518 % | Historical Best: 77.518 %


Estimator: 000 | Epoch: 003 | Batch: 000 | Loss: 0.34323 | Correct: 111/128
Estimator: 000 | Epoch: 003 | Batch: 100 | Loss: 0.29648 | Correct: 115/128
Estimator: 000 | Epoch: 003 | Batch: 200 | Loss: 0.24975 | Correct: 116/128
Estimator: 000 | Epoch: 003 | Batch: 300 | Loss: 0.31243 | Correct: 115/128
Estimator: 000 | Epoch: 003 | Batch: 400 | Loss: 0.23258 | Correct: 120/128
Estimator: 000 | Epoch: 003 | Batch: 500 | Loss: 0.31879 | Correct: 113/128
Estimator: 000 | Epoch: 003 | Batch: 600 | Loss: 0.34101 | Correct: 111/128
Estimator: 000 | Epoch: 003 | Batch: 700 | Loss: 0.31456 | Correct: 113/128
Estimator: 000 | Epoch: 003 | Batch: 800 | Loss: 0.28671 | Correct: 114/128
Estimator: 000 | Epoch: 003 | Batch: 900 | Loss: 0.30768 | Correct: 115/128
Estimator: 000 | Epoch: 003 | Batch: 1000 | Loss: 0.26017 | Correct: 117/128
Estimator: 000 | Epoch: 003 | Batch: 1100 | Loss: 0.28470 | Correct: 115/128
Estimator: 000 | Epoch: 003 | Batch: 1200 | Loss: 0.28898 | Correct: 118/128
Estimator

2021-04-02 13:32:27,916 - INFO: Epoch: 003 | Validation Acc: 77.451 % | Historical Best: 77.518 %


## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction_lstm.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        if i < len(predict) - 1 and i > 0: # post processing
            if (predict[i-1] == predict[i+1] and y != predict[i-1]):
                y = predict[i-1]
            if (predict[i-1] != predict[i+1] and y != predict[i-1] and y != predict[i+1]):
                y = predict[i-1]
        f.write('{},{}\n'.format(i, y))

In [ ]:
from google.colab import files
files.download("prediction_lstm.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Reference**

Source:

Ray Chen @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW02/HW02-1.ipynb)

https://ensemble-pytorch.readthedocs.io/en/latest/